In [ ]:
! nvidia-smi


Wed Feb  7 02:53:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#安装库
!pip install datasets
!pip install transformers[torch]
!pip install torchkeras

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("/content/news.csv")

In [ ]:
data

,label,text
0,教育,澳移民子女成长记：带着中国心融入主流社会新华网悉尼5月31日电 无论哪个国家的父辈与子女间都...
1,体育,快船vs火箭首发：休城旨在练兵 小德帕特森进先发新浪体育讯北京时间4月10日消息，在常规赛还...
2,科技,3英寸屏高清闪存DV 三洋TH1特价1499 作者：中关村在线 飘雪 ...
3,时尚,贝嫂乏味归乏味 还有人买账Victoria Beckham 乏味归乏味 还有人买账大姐大的阵...
4,房产,三亚岭南赶房集 金九银十再兴购房游纯粹的旅行闲适有余却“收获”不足，设计一条可以兼容曼妙风景...
...,...,...
995,娱乐,组图：本-斯蒂勒与布莱克助阵《寻找伴郎》首映新浪娱乐讯 北京时间3月18日(美国当地时间3月...
996,房产,"房源阶段性不足价格高涨 楼市将上演新一轮疯狂近日，中海地产(企业专区,旗下楼盘)以70.06..."
997,科技,宽屏广角高清DC！佳能110IS仅售1980【山东IT在线报道】佳能IXUS 110 IS装...
998,时政,公安部建成打拐DNA数据库通缉50名人贩●不到1个月，侦破拐卖儿童、妇女案件逾300起，解救...


In [ ]:
def label_dic(data,label):
    d = {}
    labels = data[label].unique()
    for i,v in enumerate(labels):
        d[v] = i
    return d

In [ ]:
label_dic = label_dic(data,"label")

In [ ]:
label_dic

{'教育': 0,
 '体育': 1,
 '科技': 2,
 '时尚': 3,
 '房产': 4,
 '家居': 5,
 '财经': 6,
 '时政': 7,
 '娱乐': 8,
 '游戏': 9}

In [ ]:
def get_train_data(data,col_x,col_y,label_dic):
    content = data[col_x]
    label = []
    for i in data[col_y]:
        label.append(label_dic.get(i))
    return content,label

In [ ]:
content,label  = get_train_data(data,"text","label",label_dic)

In [ ]:
from sklearn.utils import shuffle

In [ ]:
data = pd.DataFrame({"content":content,"label":label})
data = shuffle(data)

In [ ]:
from transformers import AutoTokenizer #BertTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese') #需要和模型一致

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

In [ ]:
train_len = round(len(data)*0.8)
train_data = tokenizer(data.content.to_list()[:train_len], padding = "max_length", max_length = 128, truncation=True ,return_tensors = "pt")
train_label = data.label.to_list()[:train_len]

In [ ]:
train_data

{'input_ids': tensor([[ 101, 1426, 2126,  ..., 5440, 2175,  102],
        [ 101,  676, 3613,  ..., 4638,  671,  102],
        [ 101, 1057, 7305,  ...,  924,  749,  102],
        ...,
        [ 101,  100, 3297,  ..., 2157,  832,  102],
        [ 101,  914, 7218,  ..., 8024, 2772,  102],
        [ 101, 4324, 2157,  ...,  833,  677,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=10)

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 16
train = TensorDataset(train_data["input_ids"], train_data["attention_mask"], torch.tensor(train_label))
train_sampler = RandomSampler(train)
train_dataloader = DataLoader(train, sampler=train_sampler, batch_size=batch_size)

In [ ]:
#定义优化器
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=1e-4)
#定义学习率和训练轮数
num_epochs = 1
from transformers import get_scheduler
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
for epoch in range(1):
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        if step % 10 == 0 and not step == 0:
            print("step: ",step, "  loss:",total_loss/(step*batch_size))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
        outputs = model(b_input_ids,
                    token_type_ids=None,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        lr_scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)
    print("avg_loss:",avg_train_loss)

step:  10   loss: 0.12209509238600731
step:  20   loss: 0.09063726030290127
step:  30   loss: 0.06749602984637022
step:  40   loss: 0.05555837969295681
avg_loss: 0.765844543427229


In [ ]:
inp = "专家指导：参加SSAT考试读美国优质高中(图)SSAT考试的全称是Secondary SchoolAdmission Test)，是美国(微博)中学入学测试，相当于中国的中考，近年来，越来越多的中国学生通过参加SSAT申请美国高中，然后一步步进入世界一流大学。"

In [ ]:
import numpy as np
test = tokenizer(inp,return_tensors="pt",padding="max_length",max_length=128)

In [ ]:
len(test["input_ids"][0])

128

In [ ]:
model.eval()
with torch.no_grad():
    test["input_ids"] = test["input_ids"].to(device)
    test["attention_mask"] = test["attention_mask"].to(device)
    outputs = model(test["input_ids"],
                    token_type_ids=None,
                    attention_mask=test["attention_mask"])


In [ ]:
pred_flat = np.argmax(outputs["logits"].cpu(),axis=1).numpy().squeeze()

In [ ]:
pred_flat

array(0)

In [ ]:
pred_flat.tolist()

0

In [ ]:
id2label_dic={}
for k,v in label_dic.items():
    id2label_dic[v] = k


In [ ]:
id2label_dic

{0: '教育',
 1: '体育',
 2: '科技',
 3: '时尚',
 4: '房产',
 5: '家居',
 6: '财经',
 7: '时政',
 8: '娱乐',
 9: '游戏'}

In [ ]:
id2label_dic[pred_flat.tolist()]

'教育'

In [ ]:
model.config.id2label = id2label_dic

In [ ]:
model.save_pretrained("./bert0207")
tokenizer.save_pretrained("./bert0207")

('./bert0207/tokenizer_config.json',
 './bert0207/special_tokens_map.json',
 './bert0207/vocab.txt',
 './bert0207/added_tokens.json',
 './bert0207/tokenizer.json')

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification",model="./bert0207")

In [ ]:
classifier(inp)

[{'label': '教育', 'score': 0.9345001578330994}]